<a href="https://colab.research.google.com/github/wandb/nb_helpers/blob/main/nbs/01_wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{tracker} -->

In [ ]:
#| default_exp wandb

# Weights and Biases specifics
> To deal with wandb specific usage for our examples repo https://github.com/wandb/examples

In [ ]:
#| export
import re, os
from pathlib import Path
from tempfile import TemporaryDirectory

from fastcore.script import Param, call_parse, store_true
from fastcore.basics import listify

from execnb.nbio import read_nb, write_nb

from nb_helpers.utils import (
    detect_imported_libs,
    git_main_name,
    search_string_in_nb,
    find_nbs,
    RichLogger,
    git_local_repo,
)
from nb_helpers.clean import clean_nb, clean_one
from nb_helpers.colab import add_colab_badge, add_colab_metadata, get_colab_url, has_colab_badge

In [ ]:
this_nb = Path("01_wandb.ipynb")
notebook = read_nb(this_nb)

In [ ]:
#| export
WANDB_FEATURES = "Table,sweep,WandbCallback,WandbLogger,Artifact"

In [ ]:
#| export
PYTHON_LIBS = "torch,keras,tensorflow,sklearn,yolo,jax,pandas,numpy,spacy,transformers,lightning,fastai"

In [ ]:
#| export
def get_wandb_tracker(nb):
    "Get the value inside <!--- @wandbcode{tracker} -->"
    for cell in nb["cells"]:
        if "@wandbcode" in cell["source"]:
            match = re.search(r"@wandbcode{(.*?)}", cell["source"])
            if match is not None:
                tracker_id = match.group(1)  
                return tracker_id.split(",")[0]
    return ""

In [ ]:
assert get_wandb_tracker(notebook) == "tracker"

In [ ]:
#| export
def search_code(nb, features=WANDB_FEATURES):
    "Search notebook for features"
    present_features = []
    for feat in listify(features.split(",")):
        if search_string_in_nb(nb, feat):
            present_features.append(feat)
    return present_features

In [ ]:
search_code(notebook)

['Table', 'sweep', 'WandbCallback', 'WandbLogger', 'Artifact']

In [ ]:
#| export
@call_parse
def summary_nbs(
    path: Param("A path to nb files", Path, nargs="?", opt=False) = os.getcwd(),
    wandb_features: Param("wandb features to identify, comma separated", str) = WANDB_FEATURES,
    out_file: Param("Export to csv file", Path) = "summary_table.csv",
    full_path: Param("Use full path for fname", store_true) = False,
):
    path = Path(path)
    # out_file = (path.parent / out_file).with_suffix(".csv")
    logger = RichLogger(columns=["fname", "tracker", "wandb", "python libs", "colab_idx"], out_file=out_file)

    files = find_nbs(path)
    assert len(files) > 0, "There is no `ipynb` notebooks in the path you submited"

    logger.info(f"Reading {len(files)} notebooks")

    repo_path = git_local_repo(files[0])
    branch = git_main_name(repo_path)

    for nb_path in files:
        print(nb_path)
        nb = read_nb(nb_path)
        tracker_id = get_wandb_tracker(nb)
        fname = nb_path.name if not full_path else nb_path.relative_to(repo_path)
        features = search_code(nb, wandb_features)
        libs = detect_imported_libs(nb)
        colab_cell_idx = has_colab_badge(nb)
        row = [
            str(fname),
            tracker_id,
            ", ".join(features),
            ", ".join(libs),
            str(colab_cell_idx) if colab_cell_idx != -1 else "",
        ]
        colab_link = get_colab_url(nb_path, branch)
        logger.writerow(row, colab_link)
    logger.to_table()
    logger.to_csv(Path(out_file).with_suffix(".csv"))
    logger.to_md(Path(out_file).with_suffix(".md"))

In [ ]:
with TemporaryDirectory() as d:
    summary_nbs(out_file=Path(d)/"summary.csv")

/Users/tcapelle/work/nb_helpers/nbs/00_colab.ipynb
/Users/tcapelle/work/nb_helpers/nbs/01_wandb.ipynb
/Users/tcapelle/work/nb_helpers/nbs/02_utils.ipynb
/Users/tcapelle/work/nb_helpers/nbs/03_actions.ipynb
/Users/tcapelle/work/nb_helpers/nbs/04_clean.ipynb
/Users/tcapelle/work/nb_helpers/nbs/05_run.ipynb
/Users/tcapelle/work/nb_helpers/nbs/06_export.ipynb
/Users/tcapelle/work/nb_helpers/nbs/test_data/fail_nb.ipynb
/Users/tcapelle/work/nb_helpers/nbs/test_data/test_nb.ipynb


┌───┬──────────────────┬─────────┬────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────┬───────────┐
│ # │ fname            │ tracker │ wandb                                              │ python libs                                                                    │ colab_idx │
├───┼──────────────────┼─────────┼────────────────────────────────────────────────────┼────────────────────────────────────────────────────────────────────────────────┼───────────┤
│ 0 │ ]8;id=162539;https://colab.research.google.com/github/wandb/nb_helpers/blob/main/nbs/00_colab.ipynb\00_colab.ipynb]8;;\   │         │                                                    │ pathlib, IPython, fastcore, execnb, nb_helpers, nbdev                          │ 0         │
│ 1 │ ]8;id=423445;https://colab.research.google.com/github/wandb/nb_helpers/blob/main/nbs/01_wandb.ipynb\01_wandb.ipynb]8;;\   │ tracker │ Table, sweep, WandbCallback, WandbLogger, Artifact │ re, os, pathlib, tempfile, fastcore, execnb, nb_helpers                        │ 0         │
│ 2 │ ]8;id=954902;https://colab.research.google.com/github/wandb/nb_helpers/blob/main/nbs/02_utils.ipynb\02_utils.ipynb]8;;\   │         │ Table                                              │ io, json, sys, re, csv, logging, git, types, fastcore, datetime, rich,         │ 0         │
│   │                  │         │                                                    │ pathlib, execnb                                                                │           │
│ 3 │ ]8;id=723838;https://colab.research.google.com/github/wandb/nb_helpers/blob/main/nbs/03_actions.ipynb\03_actions.ipynb]8;;\ │         │ Artifact                                           │ json, fastcore, ghapi, nb_helpers, IPython                                     │ 0         │
│ 4 │ ]8;id=730783;https://colab.research.google.com/github/wandb/nb_helpers/blob/main/nbs/04_clean.ipynb\04_clean.ipynb]8;;\   │         │ Table                                              │ io, json, os, pathlib, rich, fastcore, execnb, nb_helpers                      │ 0         │
│ 5 │ ]8;id=677755;https://colab.research.google.com/github/wandb/nb_helpers/blob/main/nbs/05_run.ipynb\05_run.ipynb]8;;\     │         │                                                    │ time, os, logging, re, pathlib, typing, fastcore, rich, execnb, nb_helpers     │ 0         │
│ 6 │ ]8;id=208771;https://colab.research.google.com/github/wandb/nb_helpers/blob/main/nbs/06_export.ipynb\06_export.ipynb]8;;\  │         │                                                    │ os, pathlib, rich, fastcore, nb_helpers                                        │ 0         │
│ 7 │ ]8;id=811731;https://colab.research.google.com/github/wandb/nb_helpers/blob/main/nbs/test_data/fail_nb.ipynb\fail_nb.ipynb]8;;\    │         │                                                    │                                                                                │ 0         │
│ 8 │ ]8;id=791519;https://colab.research.google.com/github/wandb/nb_helpers/blob/main/nbs/test_data/test_nb.ipynb\test_nb.ipynb]8;;\    │ tracker │                                                    │ os, sys, logging, pathlib, fastcore, itertools                                 │ 0         │
└───┴──────────────────┴─────────┴────────────────────────────────────────────────────┴────────────────────────────────────────────────────────────────────────────────┴───────────┘

In [ ]:
#| export
@call_parse
def fix_nbs(
    path: Param("A path to nb files", Path, nargs="?", opt=False) = os.getcwd(),
    colab_cell_idx: Param("Cell idx where the colab badge must go", int) = 0,
    branch: Param("The branch", str) = None,
):

    path = Path(path)
    files = find_nbs(path)
    assert len(files) > 0, "There is no `ipynb` notebooks in the path you submited"

    for nb_path in files:
        print(f"Add colab badge to {nb_path}")
        clean_one(nb_path, clear_outs=True)
        nb = read_nb(nb_path)
        tracker = get_wandb_tracker(nb)
        if tracker is not None and tracker != "":
            tracker = f"<!--- @wandbcode{{{tracker}}} -->"
        nb = add_colab_badge(nb, nb_path, branch=branch, idx=colab_cell_idx, tracker=tracker)
        add_colab_metadata(nb)
        clean_nb(nb)
        write_nb(nb, nb_path)

let's make sure that every notebooks has a `open in colab` badge

In [ ]:
#|eval: false
fix_nbs()

Add colab badge to /Users/tcapelle/work/nb_helpers/nbs/00_colab.ipynb
Add colab badge to /Users/tcapelle/work/nb_helpers/nbs/01_wandb.ipynb
Add colab badge to /Users/tcapelle/work/nb_helpers/nbs/02_utils.ipynb
Add colab badge to /Users/tcapelle/work/nb_helpers/nbs/03_actions.ipynb
Add colab badge to /Users/tcapelle/work/nb_helpers/nbs/04_clean.ipynb
Add colab badge to /Users/tcapelle/work/nb_helpers/nbs/05_run.ipynb
Add colab badge to /Users/tcapelle/work/nb_helpers/nbs/06_export.ipynb
Add colab badge to /Users/tcapelle/work/nb_helpers/nbs/test_data/fail_nb.ipynb
Add colab badge to /Users/tcapelle/work/nb_helpers/nbs/test_data/test_nb.ipynb


We may want to check what libs are in use:

In [ ]:
#| export
@call_parse
def filter_nbs(
    path: Param("A path to nb files", Path, nargs="?", opt=False) = os.getcwd(),
    lib_name: Param("Python lib names to filter, eg: tensorflow", str) = "",
    out_file: Param("Export to csv file", Path) = "filtered_nbs.csv",
):

    path = Path(path)

    logger = RichLogger(columns=["fname", "python libs"], out_file=out_file)

    files = find_nbs(path)
    assert len(files) > 0, "There is no `ipynb` notebooks in the path you submited"

    repo_path = git_local_repo(files[0])

    for nb_path in files:
        nb = read_nb(nb_path)
        fname = nb_path.relative_to(repo_path)
        libs = ", ".join(detect_imported_libs(nb))
        if lib_name in libs:
            row = [str(fname), libs]
            logger.writerow(row)

    logger.to_table()
    logger.to_csv(Path(out_file).with_suffix(".csv"), format_link=False)

In [ ]:
with TemporaryDirectory() as d:
    filter_nbs(out_file=Path(d)/"summary.csv")

┌───┬─────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────┐
│ # │ fname                       │ python libs                                                                            │
├───┼─────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────────┤
│ 0 │ ]8;id=855218;None\nbs/00_colab.ipynb]8;;\          │ pathlib, IPython, fastcore, execnb, nb_helpers, nbdev                                  │
│ 1 │ ]8;id=429512;None\nbs/01_wandb.ipynb]8;;\          │ re, os, pathlib, tempfile, fastcore, execnb, nb_helpers                                │
│ 2 │ ]8;id=219088;None\nbs/02_utils.ipynb]8;;\          │ io, json, sys, re, csv, logging, git, types, fastcore, datetime, rich, pathlib, execnb │
│ 3 │ ]8;id=767599;None\nbs/03_actions.ipynb]8;;\        │ json, fastcore, ghapi, nb_helpers, IPython                                             │
│ 4 │ ]8;id=566972;None\nbs/04_clean.ipynb]8;;\          │ io, json, os, pathlib, rich, fastcore, execnb, nb_helpers                              │
│ 5 │ ]8;id=231570;None\nbs/05_run.ipynb]8;;\            │ time, os, logging, re, pathlib, typing, fastcore, rich, execnb, nb_helpers             │
│ 6 │ ]8;id=678543;None\nbs/06_export.ipynb]8;;\         │ os, pathlib, rich, fastcore, nb_helpers                                                │
│ 7 │ ]8;id=247160;None\nbs/test_data/fail_nb.ipynb]8;;\ │                                                                                        │
│ 8 │ ]8;id=882075;None\nbs/test_data/test_nb.ipynb]8;;\ │ os, sys, logging, pathlib, fastcore, itertools                                         │
└───┴─────────────────────────────┴────────────────────────────────────────────────────────────────────────────────────────┘